In [1]:
from tool import *

# 1. 검색 조건 설정

In [8]:
search_year = '.'
search_month = '.'
search_date = '.'
sn = ['305QCLH563104']
model_name = '.'
responsibility = '.'
reason = '.'
supplier = '.'
judge = '.'
defect = '.'
doa = '.'

In [9]:
with open('D:/Data/PC Inspection DB.bin', 'rb') as f:
    df = pickle.load(f)

In [10]:
cond1 = df['Inspection Date'].dt.year.astype(str).str.contains(search_year, na=True)
cond2 = df['Inspection Date'].dt.month.astype(str).str.contains(search_month, na=True)
cond3 = df['Inspection Date'].dt.date.astype(str).str.contains(search_date, na=True)
cond4 = df['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model).replace(vendor_find).str.contains(supplier, na=True)
cond5 = df['Mapping Model.Suffix'].str.contains(model_name, na=True)
# cond6 = df['S/N'].str.contains(sn, na=True)
cond6 = df['S/N'].isin(sn)
cond7 = df['Responsibility'].str.contains(responsibility, na=True)
cond8 = df['Inspection_Reason'].str.contains(reason, na=True)
cond9 = df['Judge'].str.contains(judge, na=True)
cond10 = df['Defect Reason_Korean'].str.contains(defect, na=True)
cond11 = df['DOA Number'].str.contains(doa, na=True)
df_new = df[cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7 & cond8 & cond9 & cond10 & cond11]
df_new

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,DOA Number,Responsibility,Inspection_Reason,Defect Category


In [11]:
with open('D:/Data/Quanta Korea Repair list.bin', 'rb') as f:
    df = pickle.load(f)

In [15]:
df[df['S/N'].str.contains('305QCLH560128')]

,Vendor,Series,Mapping Model.Suffix,S/N,Quanta PN,Defective_Symptom(KOR),Defective_Symptom(ENG),Repair Item1,Repair Material P/N1,Repair Item2,Repair Material P/N2,Repair Item3,Repair Material P/N3,Remark


# 2. 월별 전수검사 결과 및 불량율 확인

In [5]:
df_new['Series'] = df_new['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model)
df_new.loc[:,'Vendor'] = df_new['Series'].replace(vendor_find)
df_new['DOA Number'].fillna('-', inplace=True)
df_new = pd.pivot_table(df_new, index=['Inspection Date', 'Vendor','Series', 'Inspection_Reason', 'Responsibility', 'DOA Number'], 
                        columns=['Judge'], values='S/N', aggfunc='count')
df_new = df_new.fillna(0)
df_new = df_new.convert_dtypes()
df_new['Total'] = df_new['NG'] + df_new['OK']
df_new.loc[:, 'Defective Rate(%)'] = df_new['NG'] * 100 / df_new['Total']
df_new

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_7936\1102286439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Series'] = df_new['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model)
C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_7936\1102286439.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.loc[:,'Vendor'] = df_new['Series'].replace(vendor_find)
C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_7936\1102286439.py:3: SettingWithCopyWarning: 
A value is trying 

Judge                                                                                                         NG  \
Inspection Date Vendor   Series Inspection_Reason                                  Responsibility DOA Number       
2023-04-10      Quanta   14T90R Full inspection for quality control                LG             -            0   
2023-04-12      Quanta   14T90R Full inspection for quality control                LG             -           39   
2023-04-25      Quanta   14T90R Full inspection for quality control                LG             -            1   
                         16T90R Full inspection for quality control                LG             -            5   
2023-04-26      Pegatron 17U70Q Full inspection for sampling inspection NG(PKID... LG             -            0   
2023-04-28      Quanta   14T90R Full inspection for quality control                LG             -            2   
                         16T90R Full inspection for quality control                LG             -            6   

Judge                                                                                                          OK  \
Inspection Date Vendor   Series Inspection_Reason                                  Responsibility DOA Number        
2023-04-10      Quanta   14T90R Full inspection for quality control                LG             -            70   
2023-04-12      Quanta   14T90R Full inspection for quality control                LG             -           208   
2023-04-25      Quanta   14T90R Full inspection for quality control                LG             -           185   
                         16T90R Full inspection for quality control                LG             -            85   
2023-04-26      Pegatron 17U70Q Full inspection for sampling inspection NG(PKID... LG             -           225   
2023-04-28      Quanta   14T90R Full inspection for quality control                LG             -           118   
                         16T90R Full inspection for quality control                LG             -            89   

Judge                                                                                                         Total  \
Inspection Date Vendor   Series Inspection_Reason                                  Responsibility DOA Number          
2023-04-10      Quanta   14T90R Full inspection for quality control                LG             -              70   
2023-04-12      Quanta   14T90R Full inspection for quality control                LG             -             247   
2023-04-25      Quanta   14T90R Full inspection for quality control                LG             -             186   
                         16T90R Full inspection for quality control                LG             -              90   
2023-04-26      Pegatron 17U70Q Full inspection for sampling inspection NG(PKID... LG             -             225   
2023-04-28      Quanta   14T90R Full inspection for quality control                LG             -             120   
                         16T90R Full inspection for quality control                LG             -              95   

Judge                                                                                                         Defective Rate(%)  
Inspection Date Vendor   Series Inspection_Reason                                  Responsibility DOA Number                     
2023-04-10      Quanta   14T90R Full inspection for quality control                LG             -                         0.0  
2023-04-12      Quanta   14T90R Full inspection for quality control                LG             -                   15.789474  
2023-04-25      Quanta   14T90R Full inspection for quality control                LG             -                    0.537634  
                         16T90R Full inspection for quality control                LG             -                    5.555556  
2023-04-26      Pegatron 17U70Q Full inspection f

# 2. DOA 환입 LOT의 전수검사 결과 및 불량율 확인

In [236]:
df_new = df[df['RMA Number'].notnull()]
df_new['Series'] = df_new['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model)
df_new['Vendor'] = df_new['Series'].replace(vendor_find)
df_new = pd.pivot_table(df_new, index=['Vendor', 'Series'], columns=['Judge'], values='S/N', aggfunc='count')
df_new.fillna(0, inplace=True)
df_new['Total QTY'] = df_new['NG'] + df_new['OK']
df_new['Defective Rate(%)'] = df_new['NG'] * 100 / df_new['Total QTY']
df_new

C:\Users\paul76.lee\AppData\Local\Temp/ipykernel_13488/2777058858.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Series'] = df_new['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model)
C:\Users\paul76.lee\AppData\Local\Temp/ipykernel_13488/2777058858.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Vendor'] = df_new['Series'].replace(vendor_find)


Judge              NG     OK  Total QTY  Defective Rate(%)
Vendor   Series                                           
Pegatron 13U70P   1.0    9.0       10.0          10.000000
         13U70Q   4.0   53.0       57.0           7.017544
         14U30P   1.0   26.0       27.0           3.703704
         14U390   0.0    1.0        1.0           0.000000
         17U70P   3.0   26.0       29.0          10.344828
Quanta   14T90P   9.0  114.0      123.0           7.317073
         15U40N   3.0    4.0        7.0          42.857143
         15U40Q  16.0  153.0      169.0           9.467456
         15U50P  37.0  184.0      221.0          16.742081
         15U70N   0.0    1.0        1.0           0.000000
         15U70P   6.0   36.0       42.0          14.285714
         16T90P  39.0  240.0      279.0          13.978495

# 3. 일반전수검사 vs DOA 환입품 전수검사 비교

In [4]:
df_rma_inspection = df[df['RMA Number'].notnull()]
df_rma_inspection['Series'] = df['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model)
df_normal_inspection = df[df['RMA Number'].isnull()]
df_normal_inspection['Series'] = df_normal_inspection['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model)
df_rma_inspection = pd.pivot_table(df_rma_inspection, index=['Series'], columns=['Judge'], values='S/N', aggfunc='count')
df_rma_inspection.fillna(0, inplace=True)
df_rma_inspection = df_rma_inspection.convert_dtypes()
df_rma_inspection['Sum'] = df_rma_inspection['NG'] + df_rma_inspection['OK']
df_rma_inspection.loc['Sum', :] = df_rma_inspection.sum()
df_rma_inspection['Defective Rate(%)'] = df_rma_inspection['NG'] * 100 / df_rma_inspection['Sum']


df_normal_inspection = pd.pivot_table(df_normal_inspection, index=['Series'], columns=['Judge'], values='S/N', aggfunc='count')
df_normal_inspection.fillna(0, inplace=True)
df_normal_inspection = df_normal_inspection.convert_dtypes()
df_normal_inspection['Sum'] = df_normal_inspection['NG'] + df_normal_inspection['OK']
df_normal_inspection.loc['Sum', :] = df_normal_inspection.sum()
df_normal_inspection['Defective Rate(%)'] = df_normal_inspection['NG'] * 100 / df_normal_inspection['Sum']

C:\Users\paul76.lee\AppData\Local\Temp/ipykernel_10712/1844892110.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rma_inspection['Series'] = df['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model)
C:\Users\paul76.lee\AppData\Local\Temp/ipykernel_10712/1844892110.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_normal_inspection['Series'] = df_normal_inspection['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model)


In [5]:
df_normal_inspection

,NG,OK,Sum,Defective Rate(%)
Series,,,,
11T50Q,36,1690,1726,2.085747
14T90P,102,4792,4894,2.084185
14U30P,0,40,40,0.0
15U40Q,4,269,273,1.465201
15U50P,8,330,338,2.366864
15U70P,2,482,484,0.413223
16T90P,643,22135,22778,2.822899
17U70P,3,968,971,0.30896
Sum,798,30706,31504,2.533012


In [6]:
df_rma_inspection

,NG,OK,Sum,Defective Rate(%)
Series,,,,
13U70P,1,9,10,10.0
13U70Q,4,53,57,7.017544
14T90P,9,114,123,7.317073
14U30P,1,26,27,3.703704
14U390,0,1,1,0.0
15U40N,3,4,7,42.857143
15U40Q,16,153,169,9.467456
15U50P,37,184,221,16.742081
15U70N,0,1,1,0.0


# 4. 월별 전수검사 내역/사유/귀책 정리

In [8]:
df_new = df
df_new['Series'] = df_new['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model)
df_new.loc[:,'Vendor'] = df_new['Series'].replace(vendor_find)
df_new = pd.pivot_table(df_new, index=['Vendor', 'Inspection Date', 'Series', 'Mapping Model.Suffix', 'Inspection_Reason', 'Responsibility'], columns=['Judge'], values='S/N', aggfunc='count')
df_new = df_new.fillna(0)
df_new = df_new.convert_dtypes()
df_new['Total'] = df_new['NG'] + df_new['OK']
df_new.loc[:, 'Defective Rate(%)'] = df_new['NG'] * 100 / df_new['Total']
df_new.reset_index(inplace=True)
df_new

,Vendor,Inspection Date,Series,Mapping Model.Suffix,Inspection_Reason,Responsibility,NG,OK,Total,Defective Rate(%)
0,Pegatron,2022-01-24,14U30P,14U30P-E.A32FK,Micro SD카드 Locking 불량에 따른 Lot 전수 검사,Supplier,0,40,40,0.0
1,Pegatron,2022-02-09,17U70P,17U70P-P.AA7DK,F11 초기화 시 Recovery용 저장공간 부족 문제 관련 Image재작업,LG,0,430,430,0.0
2,Pegatron,2022-02-09,17U70P,17U70P-P.AA7SK,F11 초기화 시 Recovery용 저장공간 부족 문제 관련 Image재작업,LG,0,100,100,0.0
3,Pegatron,2022-02-09,17U70P,17U70P-P.AR5SK,F11 초기화 시 Recovery용 저장공간 부족 문제 관련 Image재작업,LG,0,54,54,0.0
4,Pegatron,2022-02-10,17U70P,17U70P-P.AR5SK,F11 초기화 시 Recovery용 저장공간 부족 문제 관련 Image재작업,LG,0,46,46,0.0
...,...,...,...,...,...,...,...,...,...,...
226,Quanta,2022-04-02,15U50P,15U50P-G.AP5VLF,DOA 수리 후 환입시료 검사,LG,14,63,77,18.181818
227,Quanta,2022-04-02,15U70P,15UD70P-P.AX50K,DOA 수리 후 환입시료 검사,LG,0,1,1,0.0
228,Quanta,2022-04-02,15U40Q,15U40Q-G.AR30K,DOA 수리 후 환입시료 검사,LG,0,3,3,0.0
229,Quanta,2022-04-02,15U40Q,15UD40Q-G.AX30K,DOA 수리 후 환입시료 검사,LG,0,4,4,0.0


# 4. 일별 전수검사 S/N 개수와 중복제외 개수 확인

In [60]:
a = df.groupby('Inspection Date', dropna=False)['S/N'].unique().apply(len)
a.name = 'S/N Unique'
b = df.groupby('Inspection Date', dropna=False)['S/N'].count()
b.name = 'S/N'
c = a - b
c.name = 'Gap'
df1 = pd.concat([a,b,c], axis=1)
df1.loc['Sum'] = df1.sum()
df1

,S/N Unique,S/N,Gap
Inspection Date,,,
2022-01-06 00:00:00,990,990,0
2022-01-07 00:00:00,1330,1330,0
2022-01-10 00:00:00,1416,1416,0
2022-01-11 00:00:00,621,621,0
2022-01-12 00:00:00,592,601,-9
2022-01-13 00:00:00,408,408,0
2022-01-18 00:00:00,580,580,0
2022-01-19 00:00:00,504,517,-13
2022-01-24 00:00:00,98,98,0


In [4]:
cond = (df['Inspection Date'] == '2022-03-26')
df[cond]['S/N'].value_counts()[df[cond]['S/N'].value_counts()>1]

203QCRN575334    2
203QCAS574728    2
203QCJZ575188    2
203QCDG575538    2
203QCAS575472    2
203QCWC575905    2
203QCHE575793    2
203QCTB575681    2
203QCCV576923    2
203QCPY575360    2
203QCKJ574461    2
203QCSF574694    2
203QCUK574822    2
203QCHE574473    2
203QCQX574812    2
203QCVU574495    2
203QCNL574535    2
Name: S/N, dtype: int64

# 5. 특정 기간의 상세 전수검사 내용 확인

In [30]:
year = 2022
month = 3

In [31]:
cond = (df['Inspection Date'].dt.year == 2022) * (df['Inspection Date'].dt.month == 3)

In [38]:
df_new = df[cond]
df_new['Series'] = df_new['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model)
df_new = pd.pivot_table(df_new, index=['Inspection Date', 'Inspection_Reason', 'Series'], columns='Judge', values='S/N',aggfunc='count')
df_new = df_new.fillna(0)
df_new = df_new.convert_dtypes()
df_new['합계'] = df_new['NG'] + df_new['OK']
df_new

C:\Users\paul76.lee\AppData\Local\Temp/ipykernel_16172/3995427677.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Series'] = df_new['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model)


NG  \
Inspection Date Inspection_Reason                                  Series       
2022-03-03      F11 PC초기화 실패 입고검사 품질 Issue 관련 물류재고 137대 전량 IMAG... 17U70P   2   
                품질안정화를 위한 전수검사                                     14T90P  11   
                                                                   16T90P   2   
2022-03-05      DC Mode 전원무감, Speaker Sound 좌/우 편차 관련 전수 검사        15U40Q   4   
                Type-C 인식 무감 관련 물류 재고 40대 추가 검증                    15U50P   0   
                품질안정화를 위한 전수검사                                     16T90P  87   
2022-03-07      품질 안정화를 위한 전수 검사                                   16T90P  33   
2022-03-12      품질 안정화를 위한 전수 검사                                   14T90P   6   
                                                                   16T90P  38   
2022-03-14      품질 안정화를 위한 전수 검사                                   14T90P   0   
                                                                   16T90P  29   
2022-03-16      A-Part 엣지부 다이어컷 부위 색상 상이 이슈 재선별 양품화                16T90P   0   
                품질안정화를 위한 전수검사                                     16T90P  10   
2022-03-18      품질 안정화를 위한 전수 검사                                   16T90P   9   
2022-03-23      품질 안정화를 위한 전수 검사                                   16T90P  20   
2022-03-24      품질 안정화를 위한 전수 검사                                   16T90P   2   
2022-03-25      충남교육청 납기준수를 위한 불량 41대 양품화 작업                       16T90P   0   
2022-03-26      품질 안정화를 위한 전수 검사                                   16T90P  19   
2022-03-27      품질 안정화를 위한 전수 검사                                   16T90P   8   
2022-03-28      충남교육청 납기준수를 위한 불량 167대 양품화 작업                      16T90P   0   
                품질안정화를 위한 전수검사                                     16T90P   0   
2022-03-30      품질 안정화를 위한 전수 검사                                   14T90P   1   

                                                                             OK  \
Inspection Date Inspection_Reason                                  Series         
2022-03-03      F11 PC초기화 실패 입고검사 품질 Issue 관련 물류재고 137대 전량 IMAG... 17U70P   135   
                품질안정화를 위한 전수검사                                     14T90P   457   
                                                                   16T90P    58   
2022-03-05      DC Mode 전원무감, Speaker Sound 좌/우 편차 관련 전수 검사        15U40Q   269   
                Type-C 인식 무감 관련 물류 재고 40대 추가 검증                    15U50P    40   
                품질안정화를 위한 전수검사                                     16T90P  1256   
2022-03-07      품질 안정화를 위한 전수 검사                                   16T90P  2038   
2022-03-12      품질 안정화를 위한 전수 검사                                   14T90P   228   
                                                                   16T90P  1132   
2022-03-14      품질 안정화를 위한 전수 검사                                   14T90P    60   
                                                                   16T90P  1118   
2022-03-16      A-Part 엣지부 다이어컷 부위 색상 상이 이슈 재선별 양품화                16T90P    52   
                품질안정화를 위한 전수검사                                     16T90P   257   
2022-03-18      품질 안정화를 위한 전수 검사                                   16T90P   387   
2022-03-23      품질 안정화를 위한 전수 검사                                   16T90P   500   
2022-03-24      품질 안정화를 위한 전수 검사                                   16T90P    99   
2022-03-25      충남교육청 납기준수를 위한 불량 41대 양품화 작업                       16T90P    41   
2022-03-26      품질 안정화를 위한 전수 검사                                   16T90P  1381   
2022-03-27      품질 안정화를 위한 전수 검사                                   16T90P   786   
2022-03-28      충남교육청 납기준수를 위한 불량 167대 양품화 작업                      16T90P   167   
                품질안정화를 위한 전수검사                                     16T90P   169   
2022-03-30      품질 안정화를 위한 전수 검사                                   14T90P    99   

                                                                             합계  
Inspection Date Inspection_Reason                      

## ---------------------- 주의) 하기는 데이타 저장할 때만 사용할 것 -----------------------------

In [13]:
df.shape # 저장할 데이타의 크기를 항상 확인하자!

(100006, 12)

In [14]:
with open('D:/Data/PC Inspection DB.bin', 'wb') as f:
    pickle.dump(df, f)